# Breast Cancer Diagnostics

Using breast cancer data from University of Wisconsin Hospitals, Madison, I will attempt to fit a supervised learning model to predict whether tumors are benign or malignant. The features are all integers between 1 and 10 that represent nine measurements of the tumor tissue. The class outcome variable gives a value of 2 for benign and 4 for malignant

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('wi_breast_cancer.txt')
df.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [3]:
# Change class to 0 for benign, 1 for malignant
df['Class'] = df['Class'].apply(lambda x: int((x//2)-1))
df.head(20)

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0
5,1017122,8,10,10,8,7,10,9,7,1,1
6,1018099,1,1,1,1,2,10,3,1,1,0
7,1018561,2,1,2,1,2,1,3,1,1,0
8,1033078,2,1,1,1,2,1,1,1,5,0
9,1033078,4,2,1,1,2,1,2,1,1,0


In [4]:
# Check mean and variance
df.describe()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bland Chromatin,Normal Nucleoli,Mitoses,Class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,0.344778
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.475636
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,0.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,0.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,1.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000


In [5]:
# Check for null values
df.isna().sum()

Sample code number             0
Clump Thickness                0
Uniformity of Cell Size        0
Uniformity of Cell Shape       0
Marginal Adhesion              0
Single Epithelial Cell Size    0
Bare Nuclei                    0
Bland Chromatin                0
Normal Nucleoli                0
Mitoses                        0
Class                          0
dtype: int64

In [18]:
# Drop rows that are missing values for Bare Nuclei
df = df.drop(df.index[np.where(df['Bare Nuclei']=='?')])
len(df)

683

In [37]:
# Make feature set, outcome variable, and train/test split
X = df.iloc[:, 1:-1]
Y = df.iloc[:, -1]
X_train, X_test = df.iloc[:342, 1:-1], df.iloc[342:, 1:-1]
Y_train, Y_test = df.iloc[:342, -1], df.iloc[342:, -1]

Y_train.describe()

count    342.000000
mean       0.461988
std        0.499283
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Class, dtype: float64

In [25]:
# Check the mean of the train/test split
Y_test.describe()

count    341.000000
mean       0.237537
std        0.426199
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: Class, dtype: float64

In [28]:
# Try a simple Naive Bayes model
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(X_train, Y_train)
print(bnb.score(X_train, Y_train))

0.5380116959064327


In [29]:
from sklearn.metrics import confusion_matrix
pred = bnb.predict(X_test)

confusion_matrix(Y_test, pred)

array([[260,   0],
       [ 81,   0]], dtype=int64)

## Naive Bayes 
The Naive Bayes classifier was not accurate. It ended up just choosing the dominant class. Let's try some other classifiers.

In [35]:
from sklearn import ensemble

rfc = ensemble.RandomForestClassifier(n_estimators=500, class_weight='balanced')
rfc.fit(X_train, Y_train)
rfc.score(X_train, Y_train)

1.0

## Random Forest Classifier
The Random Forest Classifier looks like it has overfit, but let's run some cross-validation and do a little digging to see if that's the case.

In [40]:
from sklearn.model_selection import cross_val_score

cross_val_score(rfc, X, Y, cv=10)


array([0.94202899, 0.97101449, 0.95652174, 0.94202899, 0.98529412,
       0.97058824, 0.98529412, 0.98529412, 0.97058824, 0.98507463])

In [36]:
pred = rfc.predict(X_test)
confusion_matrix(Y_test, pred)

array([[257,   3],
       [  2,  79]], dtype=int64)

The high level of accuracy still holds up in cross-validation. Still, this is a case where we should really try to reduce the number of false negatives to 0 if at all possible. Right now that number is at two, but that is on the test set. I want to see if this holds up when checking type II error in cross-validation, the model may be worth optimizing.

In [38]:
import math

cv = 8
n = math.ceil(len(Y)/cv)
for i in range(cv):
    indices = np.random.randint(len(Y), size=n).tolist()
    rfc.fit(X.iloc[indices], Y.iloc[indices])
    pred = rfc.predict(X.drop(X.index[indices]))
    cm = confusion_matrix(Y.drop(Y.index[indices]), pred)
    print('CV Set {} Type I Error: {:2f} Type II Error: {:2f}'.format(i+1, 
                                                               (cm[0][1]/sum(cm[0])),
                                                               (cm[1][0]/sum(cm[1]))))

CV Set 1 Type I Error: 0.020513 Type II Error: 0.090047
CV Set 2 Type I Error: 0.023316 Type II Error: 0.074419
CV Set 3 Type I Error: 0.030227 Type II Error: 0.073892
CV Set 4 Type I Error: 0.020513 Type II Error: 0.057416
CV Set 5 Type I Error: 0.028278 Type II Error: 0.032864
CV Set 6 Type I Error: 0.040302 Type II Error: 0.009756
CV Set 7 Type I Error: 0.027848 Type II Error: 0.039024
CV Set 8 Type I Error: 0.030848 Type II Error: 0.071429


The type II error is higher than I would feel comfortable with for a cancer diagnostic model. I'm going to try a Gradient Boosting model.

In [48]:
# We'll make 500 iterations, use 2-deep trees, and set our loss function.
params = {'n_estimators': 500,
          'max_depth': 2,
          'loss': 'deviance'}

# Initialize and fit the model.
clf = ensemble.GradientBoostingClassifier(**params)
clf.fit(X_train, Y_train)

predict_train = clf.predict(X_train)
predict_test = clf.predict(X_test)

# Accuracy tables.
table_train = pd.crosstab(Y_train, predict_train, margins=True)
table_test = pd.crosstab(Y_test, predict_test, margins=True)

train_tI_errors = table_train.loc[0.0,1.0] / table_train.loc['All','All']
train_tII_errors = table_train.loc[1.0,0.0] / table_train.loc['All','All']

test_tI_errors = table_test.loc[0.0,1.0]/table_test.loc['All','All']
test_tII_errors = table_test.loc[1.0,0.0]/table_test.loc['All','All']

print((
    'Training set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}\n\n'
    'Test set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}'
).format(train_tI_errors, train_tII_errors, test_tI_errors, test_tII_errors))


Training set accuracy:
Percent Type I errors: 0.0
Percent Type II errors: 0.0

Test set accuracy:
Percent Type I errors: 0.011730205278592375
Percent Type II errors: 0.011730205278592375


## Gradient Boosting Model

The type II error is really low on this model, I would like to run a grid search on different parameters to optimize for minimal type II error.

In [66]:
from sklearn.metrics import recall_score, make_scorer
from sklearn.model_selection import GridSearchCV

total_recall = make_scorer(recall_score)

param_dict = {'n_estimators': [100, 250, 500], 'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.25],
              'max_depth': [2, 3, 4, None], 'loss': ['deviance', 'exponential'], 
              'subsample': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}

gscv = GridSearchCV(clf, param_dict, scoring=total_recall, iid=False, cv=6)
gscv.fit(X, Y)
gscv.score(X, Y)

1.0

In [67]:
pred = gscv.predict(X_test)
confusion_matrix(Y_test, pred)

array([[259,   1],
       [  0,  81]], dtype=int64)

## Grid Search 
After searching through many permutations of parameters, we got down to 0 false negatives in predicting the test set. Let's see exactly which parameters got us that level of accuracy and what the mean accuracy looked like.

In [68]:
gscv.best_params_

{'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': None,
 'n_estimators': 500,
 'subsample': 0.2}

In [69]:
grid_score_df = pd.DataFrame(gscv.cv_results_)
grid_score_df.head()

C:\Users\Jeff\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Jeff\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Jeff\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Jeff\Anaconda3\lib\site-packages\sklearn\utils\de

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_loss,param_max_depth,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,mean_train_score,std_train_score
0,0.112769,0.020346,0.003664,4.710346e-04,0.0001,deviance,2,100,0.1,"{'learning_rate': 0.0001, 'loss': 'deviance', ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.112769,0.006401,0.003833,3.732517e-04,0.0001,deviance,2,100,0.2,"{'learning_rate': 0.0001, 'loss': 'deviance', ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.115101,0.003075,0.003998,5.462856e-07,0.0001,deviance,2,100,0.3,"{'learning_rate': 0.0001, 'loss': 'deviance', ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.125425,0.007818,0.003336,4.763335e-04,0.0001,deviance,2,100,0.4,"{'learning_rate': 0.0001, 'loss': 'deviance', ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.122770,0.002597,0.003831,3.718725e-04,0.0001,deviance,2,100,0.5,"{'learning_rate': 0.0001, 'loss': 'deviance', ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
pd.options.display.max_columns=None
grid_score_df = grid_score_df.sort_values('rank_test_score')
grid_score_df.head(30)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_loss,param_max_depth,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,mean_train_score,std_train_score
821,0.481669,0.031258,0.006226,0.006580,0.1,exponential,4,250,0.3,"{'learning_rate': 0.1, 'loss': 'exponential', ...",0.925,1.000,0.975,0.950,0.950,1.000000,0.966667,0.027639,1,0.989950,0.994975,1.000000,0.994975,0.994975,0.995,0.994979,0.002901
948,0.281228,0.015624,0.000000,0.000000,0.25,deviance,None,100,0.4,"{'learning_rate': 0.25, 'loss': 'deviance', 'm...",0.925,1.000,0.975,0.925,0.975,1.000000,0.966667,0.031180,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000,1.000000,0.000000
1044,0.547171,0.018055,0.002604,0.005823,0.25,exponential,4,500,0.1,"{'learning_rate': 0.25, 'loss': 'exponential',...",0.925,1.000,0.975,0.925,0.975,1.000000,0.966667,0.031180,1,0.984925,1.000000,0.989950,0.994975,0.989950,0.990,0.991633,0.004735
829,0.889538,0.308107,0.004001,0.004476,0.1,exponential,4,500,0.2,"{'learning_rate': 0.1, 'loss': 'exponential', ...",0.925,1.000,0.975,0.950,0.950,1.000000,0.966667,0.027639,1,0.994975,0.989950,1.000000,0.994975,0.994975,0.995,0.994979,0.002901
1062,0.330705,0.016676,0.005208,0.007366,0.25,exponential,None,250,0.1,"{'learning_rate': 0.25, 'loss': 'exponential',...",0.900,0.975,0.975,0.975,0.975,1.000000,0.966667,0.031180,1,0.984925,0.974874,0.989950,0.989950,0.989950,0.980,0.984941,0.005788
855,0.798966,0.180946,0.003011,0.006733,0.1,exponential,None,500,0.1,"{'learning_rate': 0.1, 'loss': 'exponential', ...",0.900,1.000,0.975,0.950,0.975,1.000000,0.966667,0.034359,1,0.979899,0.974874,0.984925,0.989950,0.984925,0.980,0.982429,0.004803
748,0.860068,0.034614,0.002604,0.005822,0.1,deviance,None,500,0.2,"{'learning_rate': 0.1, 'loss': 'deviance', 'ma...",0.900,1.000,0.975,0.950,0.975,1.000000,0.966667,0.034359,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000,1.000000,0.000000
856,1.018294,0.124042,0.002604,0.005823,0.1,exponential,None,500,0.2,"{'learning_rate': 0.1, 'loss': 'exponential', ...",0.925,1.000,0.975,0.925,0.975,1.000000,0.966667,0.031180,1,0.994975,0.994975,1.000000,0.994975,0.994975,1.000,0.996650,0.002369
812,0.194470,0.011065,0.000666,0.001489,0.1,exponential,4,100,0.3,"{'learning_rate': 0.1, 'loss': 'exponential', ...",0.925,1.000,0.975,0.925,0.950,1.000000,0.962500,0.031458,9,0.984925,0.989950,1.000000,0.989950,0.994975,0.995,0.992466,0.004813
1029,0.198951,0.010173,0.000666,0.001490,0.25,exponential,4,100,0.4,"{'learning_rate': 0.25, 'loss': 'exponential',...",0.900,1.000,0.975,0.950,0.950,1.000000,0.962500,0.034611,9,0.994975,0.994975,1.000000,1.000000,1.000000,1.000,0.998325,0.002369


## Conclusion 
While the predictions on the test set showed no false negatives, the cross-validaton test scores for the top ranked parameters averaged out to 97%. While that is good, this accuracy is based on the scoring for recall (tp/tp+fn), so that all the error is derived from the false negatives. I think the model would benefit from more data.